<a href="https://colab.research.google.com/github/Faizan-404/plant-pathology/blob/main/Faizan_plant_pathology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/MyDrive/plant_pathology512and256.zip' -d '/content/image_small/'

Archive:  /content/drive/MyDrive/plant_pathology512and256.zip
replace /content/image_small/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!unzip '/content/image_small/plant_pathology_small_256.zip' -d '/content/image_folder'

In [ ]:
from fastai.vision import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
import shutil
from fastai.callbacks import SaveModelCallback

In [ ]:
pip install fastai==1.0.61

In [ ]:
path = Path('/content/image_small')

In [ ]:
train = pd.read_csv(path/'train.csv')
test = pd.read_csv(path/'test.csv')

In [ ]:
train.head()

## As can been seen here so it is one hot encoded.

In [ ]:
print('Count plot of different Features')
plt.figure(1,figsize=(16,8))
plt.subplot(221)
sns.countplot(x=train.healthy)
plt.subplot(222)
sns.countplot(x=train.multiple_diseases)
plt.subplot(223)
sns.countplot(x=train.rust)
plt.subplot(224)
sns.countplot(x=train.scab)
plt.show()

## These are quite big images

## New Image size

In [ ]:
cols = train.columns.tolist()[1:];cols

In [ ]:
tfms = get_transforms(do_flip=True)

In [ ]:
tfms

In [ ]:
#ts = (ImageList.from_df(test,path,folder='images',suffix='.jpg',cols='image_id'))

In [ ]:
data = (ImageList.from_df(train,path='/content',folder='image_folder',suffix='.jpg',cols='image_id')
                .split_by_rand_pct(0.15,seed=23)
                .label_from_df(cols=cols,label_cls = MultiCategoryList, one_hot = True)
                .transform(size=128)
                #.add_test(ts)
                .databunch(bs=64).normalize(imagenet_stats))

In [ ]:
data.show_batch(rows=3,figsize=(20,8))

In [ ]:
len(data.train_ds),len(data.valid_ds),data.train_ds[0][0].shape

In [ ]:
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_score = partial(fbeta, thresh=0.2)

In [ ]:
learn = cnn_learner(data, models.resnet50 , metrics = [acc_02, f_score])

In [ ]:

import gc
gc.collect()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
np.linspace(1e-2,1e-1,5)

In [ ]:
learn.fit_one_cycle( 10 , slice(1e-1/2) , moms = (0.8,0.7) ,callbacks=[SaveModelCallback(learn,monitor='fbeta')])

In [ ]:
img = open_image('/content/image_folder/Test_0.jpg')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
learn.export('apple_leaf_model.pkl')

In [ ]:
!cp '/content/models/bestmodel.pth' '/content/drive/MyDrive'

In [ ]:
def classify_appleleaf(img_path):
    lbls = ['healthy', 'multiple_diseases', 'rust', 'scab']
    img = open_image(img_path)
    img.show(figsize=(8,8))
    a = learn.predict(img)
    print(lbls[a[2].argmax()])

In [ ]:
classify_appleleaf('/content/image_folder/Test_1007.jpg')